In [ ]:
# ! pip install fuzzywuzzy

In [2]:
import pymysql
import json
import requests
import jellyfish
from datetime import datetime
import re
from difflib import SequenceMatcher as SM
import pandas as pd
from fuzzywuzzy import fuzz

In [33]:
timeout = 10
connection = pymysql.connect(
  charset="utf8mb4",
  connect_timeout=timeout,
  cursorclass=pymysql.cursors.DictCursor,
  db="iia_group2",
  host="world-athletics-world-athletics.h.aivencloud.com",
  password="AVNS_q4qfMZEYhnIhreAFwYi",
  read_timeout=timeout,
  port=24936,
  user="avnadmin",
  write_timeout=timeout,
)


In [17]:
def matcher(col1,col_list):
    m_score = -1
    m_col = None
    for i in col_list:
        # sim=SM(isjunk=None, a=col1, b=i).ratio()
        sim = fuzz.token_set_ratio(col1, i)
        if sim>m_score:
            m_score = sim
            m_col = i
    if(m_score/100 < 0.4):
        return None, m_score
    return m_col, m_score


In [5]:
cursor = connection.cursor()
cursor.execute("desc GST_Payers_Info_PerMonth_NS")
x = cursor.fetchall()
sql_fields = [i["Field"] for i in x]
cursor.close()

In [6]:
print(sql_fields)

['id', 'state', 'datee', 'return_type', 'eligible_payers', 'registered_payers']


In [7]:
df1 = pd.read_csv("trans_register_payers_per_month_data.csv",index_col=0)

In [8]:
df1

,state,date,Gst ( goods and service tax ) payers registered,Payer eligible for gst ( goods and service tax ) registration,Gst ( goods and service tax ) return type
0,chandigarh,2021-02-01,12708.0,12710.0,GSTR-3
1,sikkim,2019-12-01,6056.0,7450.0,GSTR-1
2,sikkim,2019-12-01,6486.0,7450.0,GSTR-3
3,sikkim,2020-01-01,2761.0,3908.0,GSTR-1
4,jammu and kashmir,2017-07-01,45971.0,52455.0,GSTR-1
...,...,...,...,...,...
4243,the dadra and nagar haveli and daman and diu,2022-03-01,13594.0,14581.0,GSTR-3
4244,the dadra and nagar haveli and daman and diu,2022-04-01,10632.0,11569.0,GSTR-1
4245,the dadra and nagar haveli and daman and diu,2022-04-01,10572.0,11569.0,GSTR-3
4246,the dadra and nagar haveli and daman and diu,2022-05-01,10308.0,11640.0,GSTR-1


In [9]:
schema_incoming_1 = list(df1.keys())

In [19]:
schema_map_1 = {}

for i in schema_incoming_1:
        schema_map_1[i] = matcher(i,sql_fields)[0]
        # print(schema_map_1[i])
        if(schema_map_1[i] == None):
                del schema_map_1[i]


In [20]:
schema_map_1

{'state': 'state',
 'date': 'datee',
 'Gst ( goods and service tax ) payers registered': 'registered_payers',
 'Gst ( goods and service tax ) return type': 'return_type'}

In [21]:
df1

,state,date,Gst ( goods and service tax ) payers registered,Payer eligible for gst ( goods and service tax ) registration,Gst ( goods and service tax ) return type
0,chandigarh,2021-02-01,12708.0,12710.0,GSTR-3
1,sikkim,2019-12-01,6056.0,7450.0,GSTR-1
2,sikkim,2019-12-01,6486.0,7450.0,GSTR-3
3,sikkim,2020-01-01,2761.0,3908.0,GSTR-1
4,jammu and kashmir,2017-07-01,45971.0,52455.0,GSTR-1
...,...,...,...,...,...
4243,the dadra and nagar haveli and daman and diu,2022-03-01,13594.0,14581.0,GSTR-3
4244,the dadra and nagar haveli and daman and diu,2022-04-01,10632.0,11569.0,GSTR-1
4245,the dadra and nagar haveli and daman and diu,2022-04-01,10572.0,11569.0,GSTR-3
4246,the dadra and nagar haveli and daman and diu,2022-05-01,10308.0,11640.0,GSTR-1


In [22]:
for i in df1.keys():
    try:
        df1[schema_map_1[i]] = df1[i]
    except KeyError:
        continue

In [23]:
df_1 = df1[list(schema_map_1.values())]

In [26]:
primary_keys = ["state","datee"]

In [27]:
schema_map_1

{'state': 'state',
 'date': 'datee',
 'Gst ( goods and service tax ) payers registered': 'registered_payers',
 'Gst ( goods and service tax ) return type': 'return_type'}

In [34]:
insert = {}
update = {}
from tqdm import tqdm
for i in sql_fields[1:]:
    insert[i] = []
    update[i] = []
print(schema_map_1.values())
cursor = connection.cursor()
for  _, i in tqdm(df_1.iterrows()):
    cursor.execute(
        "select %s from GST_Payers_Info_PerMonth_NS where state = %s AND datee = %s ",
        (
            ','.join(list(schema_map_1.values())),
            i["state"],
            i["datee"]
            )
    )
    x = cursor.fetchall()
    if(len(x) == 0):
        for j in insert.keys():
            try:
                insert[j].append(i[j])
            except KeyError:
                insert[j].append(None)
    else:
        x = x[0]
        for j in insert.keys():
            try:
                if(x[j] == None):
                    update[j].append(i[j])
                elif(x[j] != i[j]):
                    update[j].append(i[j])
                else:
                    update[j].append(None)
            except KeyError:
                update[j].append(None)
cursor.close()

dict_values(['state', 'datee', 'registered_payers', 'return_type'])


0it [00:00, ?it/s]

4248it [03:07, 22.70it/s]


In [36]:
pd.DataFrame(insert)

,state,datee,return_type,eligible_payers,registered_payers
0,chandigarh,2021-02-01,GSTR-3,None,12708.0
1,sikkim,2019-12-01,GSTR-1,None,6056.0
2,sikkim,2019-12-01,GSTR-3,None,6486.0
3,sikkim,2020-01-01,GSTR-1,None,2761.0
4,jammu and kashmir,2017-07-01,GSTR-1,None,45971.0
...,...,...,...,...,...
4243,the dadra and nagar haveli and daman and diu,2022-03-01,GSTR-3,None,13594.0
4244,the dadra and nagar haveli and daman and diu,2022-04-01,GSTR-1,None,10632.0
4245,the dadra and nagar haveli and daman and diu,2022-04-01,GSTR-3,None,10572.0
4246,the dadra and nagar haveli and daman and diu,2022-05-01,GSTR-1,None,10308.0


In [37]:
for _, i in pd.DataFrame(insert).iterrows():
    try:
        cursor = connection.cursor()
        cursor.execute(
            "insert into GST_Payers_Info_PerMonth_NS ("+', '.join(sql_fields[1:])+") values ("+ ', '.join(["%s" for i in sql_fields[1:]]) +")",
            tuple(i.to_list())
        )
        connection.commit()
        cursor.close()
    except:
        continue

for _, i in pd.DataFrame(update).iterrows():
    try:
        cursor = connection.cursor()
        final  = []
        final_tup = []
        for j in i.keys():
            if(i[j] != None):
                final.append(' '.join([str(j),"=","%s"]))
                final_tup.append(i[j])
        final = ', '.join(final)
        final_tup.extend([i["state"],i["datee"]])
        # print(final_tup)
        cursor.execute(
            f"update GST_Payers_Info_PerMonth_NS set {final} where state = %s datee = %s",
            tuple(final_tup)
        )
        connection.commit()
        cursor.close()
    except:
        continue
